In [1]:
import warnings
warnings.filterwarnings("ignore")
import xarray as xr
import netCDF4 as nc4
import numpy as np
import shapely.vectorized
import time
import pandas as pd
import geopy as gpy
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import cartopy as cart
import cartopy.crs as ccrs
import pathlib as pl
import cmocean
from scipy.spatial import cKDTree
from shapely import Polygon, box

In [2]:
mod_file = ('C:/Users/soele/OneDrive - George Mason University - O365 Production/Documents/FINESST/SWOT_Data/fort63/withalloffset/fort.63.nc')
mod_ds = nc4.Dataset(mod_file)
mod_lat = mod_ds.variables["y"][:].data
mod_lon = mod_ds.variables["x"][:].data
ssh = mod_ds.variables["zeta"]
#ssh_indice = np.where (((mod_lat > 7) & (mod_lat < 50)) & ((mod_lon < -60) & (mod_lon > -100)))[0]
#ssh_indice = np.where (((mod_lat > 32) & (mod_lat < 42)) & ((mod_lon < -72) & (mod_lon > -78)))[0]
#print(ssh[:,ssh_indice])
##normalize = mpl.colors.Normalize(vmin=np.min(ssh[:,ssh_indice]), vmax=np.max(ssh[:,ssh_indice]))
#print(normalize)
#colors = [cmap(normalize(d)) for d in ssh[:,ssh_indice]]
#print(colors)

In [3]:
start_date = pd.Timestamp('2014-07-01 00:00:00')
end_date = pd.Timestamp('2014-07-03 23:00:00')
time = pd.date_range(start_date, end_date, freq='H')

In [4]:
#Specify the Figure
root = pl.Path(r'C:/Users/soele/OneDrive - George Mason University - O365 Production/Documents/FINESST/SWOT_Data/fort63/withalloffset')
proj = ccrs.PlateCarree()
ssh_indice = np.where (((mod_lat > 7) & (mod_lat < 50)) & ((mod_lon < -60) & (mod_lon > -100)))[0]
#ssh_indice = np.where (((mod_lat > 32) & (mod_lat < 42)) & ((mod_lon < -72) & (mod_lon > -78)))[0]
cmap = cmocean.cm.thermal
normalize = mpl.colors.Normalize(vmin=np.min(ssh[:,ssh_indice]), vmax=np.max(ssh[:,ssh_indice]))
colors = [cmap(normalize(d)) for d in ssh[:,ssh_indice]]
for i in range(len(time)):
    fig  = plt.figure(figsize=(8,8),dpi=300)
    ax = plt.axes(projection = proj)
    #ax = plot_background(ax)
    ##########
    PlotZ = ssh[i,:]
    triang = tri.Triangulation(mod_lon, mod_lat, triangles=mod_ds.variables['element'][:,:]-1)
    if PlotZ.mask.any():
        point_mask_indices = np.where(PlotZ.mask)
        tri_mask = np.any(np.in1d(mod_ds.variables['element'][:,:]-1, point_mask_indices).reshape(-1, 3), axis=1)
        triang.set_mask(tri_mask)
    ts = pd.to_datetime(str(time[i]))
    title = ts.strftime('%Y-%m-%d - %H:%M')
    levels = np.arange(-1.5,2.5,0.1)
    ax.set_extent([-78, -72, 32, 42]) 
    ax.set_title(f'{title}')
    ax.coastlines(resolution='10m',linewidth=0.5)
    ax.add_feature(cart.feature.LAKES)
    ax.add_feature(cart.feature.RIVERS,linestyle="-")
    ax.add_feature(cart.feature.BORDERS.with_scale("10m"),linestyle="-")
    ax.stock_img()
    cf = ax.tricontourf(triang, PlotZ, transform = proj, c=colors, cmap = cmap, extend = 'both')
    # Colorbar junk
    cax, _ = mpl.colorbar.make_axes(ax,shrink=.9)
    cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap, norm=normalize)
    cbar.set_label(label = 'WL (m)',fontsize = '12')
    gl = ax.gridlines(draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle='--')

    # Customize the gridlines
    gl.xlabels_top = False  # Hide the top x-axis labels
    gl.ylabels_right = False  # Hide the right y-axis labels
    gl.xlabel_style = {'size': 8}  # Set the x-axis label font size
    gl.ylabel_style = {'size': 8}  # Set the y-axis label font size
    savename = ts.strftime('%Y.%m.%d-%H.%M')
    title = str(root / f'{savename}.png')
    fig.savefig(title)
    plt.close()